In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from utils import process_text
import pickle
import pandas as pd
import numpy as np

In [2]:
# filename
TFIDF = "tdidf.pkl"
MODEL = "model.pkl"

In [3]:
# load csv
all_df = pd.read_csv("data/data.csv")
test_df = pd.read_csv("data/test.csv")

In [4]:
# divide all_df into train_df(85%) and valid_df(15%)
train_df, valid_df = train_test_split(all_df, test_size=0.15, random_state=1412)
train_df = train_df.reset_index(drop=True)
valid_df = valid_df.reset_index(drop=True)

In [5]:
# y
y_train = train_df["category"]
y_valid = valid_df["category"]
y_test = test_df["category"]

In [6]:
# text faetures
# Create Vectorizer and dump to pkl
tfidf = TfidfVectorizer(
    tokenizer=process_text, ngram_range=(1, 2), min_df=20, sublinear_tf=True
)
tfidf_fit = tfidf.fit(all_df["texts"])
pickle.dump(tfidf_fit, open(TFIDF, "wb"))
print("Dump ", TFIDF, " completed")

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


Dump  tdidf.pkl  completed


In [7]:
# transform word to vector(vectorize)
text_train = tfidf_fit.transform(train_df["texts"])
text_valid = tfidf_fit.transform(valid_df["texts"])
text_test = tfidf_fit.transform(test_df["texts"])
print(text_train.shape, text_valid.shape)

(19848, 4743) (3503, 4743)


In [8]:
# convert to array
X_train = text_train.toarray()
X_valid = text_valid.toarray()
X_test = text_test.toarray()

In [9]:
# fit logistic regression models
model = LogisticRegression(
    C=2.0, penalty="l2", solver="liblinear", dual=False, multi_class="ovr"
)
model.fit(X_train, y_train)

LogisticRegression(C=2.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [10]:
# dump model to model.pickle
pickle.dump(model, open(MODEL, "wb"))
print("Dump ", MODEL, " completed")

Dump  model.pkl  completed


In [11]:
# test score
print("score_validate:", model.score(X_valid, y_valid))
print("score_test:", model.score(X_test, y_test))

score_validate: 0.7479303454182129
score_test: 0.7331555986427533


In [12]:
# Test prediction
text_predict = tfidf_fit.transform(["วันนี้อากาศดีมากเลย"])
model.predict(text_predict)

array(['pos'], dtype=object)